**Equipo OW?**
* Amilcar Gilberto Pérez Canto
* Osmar Gael Sánchez Loeza


### Requirements

In [ ]:
!pip install langchain
!pip install --upgrade --quiet langchain-core langchain-google-vertexai
!pip install chromadb
!pip install datasets
!pip install ragas
!pip install google-cloud-aiplatform google-auth > /dev/null
!pip install unstructured
!pip install unidecode

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 973.7/973.7 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 308.5/308.5 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.4/121.4 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.5/142.5 kB 1.1 MB/s eta 0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging 24.0
    Uninstalling packaging-24.0:
      Successfully uninstalled packaging-24.0
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.7/57.7 kB 761.8 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.6/125.6 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.0/139.0 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 526.8/526.8 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
CHROMA_PATH = "chroma"
DATA_PATH = "/content/drive/MyDrive/Amilcar Gilberto Pérez Canto/Proyecto_OW?/Archivos/"
EMBEDDING_MODEL = 'text-multilingual-embedding-preview-0409'
LLM_MODEL = 'gemini-1.5-pro-preview-0409'
EVAL_MODEL = "chat-bison-32k"
DICT_MEDLINE_PATH = '/content/drive/MyDrive/Amilcar Gilberto Pérez Canto/Proyecto_OW?/medlineDict.pkl'
CLOUD_PROJECT = "ragas-py"

import os
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = '/content/drive/MyDrive/Amilcar Gilberto Pérez Canto/Proyecto_OW?/ragas-py-1838c7ef18f7.json'

# LLM 🗣

## Imports

In [ ]:
from langchain_google_vertexai import ChatVertexAI, VertexAIEmbeddings
from ragas.llms import LangchainLLMWrapper
from langchain_core.prompts import ChatPromptTemplate
import ragas.metrics
from langchain_google_vertexai import HarmBlockThreshold, HarmCategory
from ragas.metrics import (
    answer_relevancy,
    context_recall,
    context_precision,
)
from ragas.metrics.critique import harmfulness

## Code

In [ ]:
def setEvalMetrics(creds, eval_model_name: str, embedding_model) -> list:
    metrics = [
        answer_relevancy,
        context_recall,
        context_precision,
        harmfulness,
    ]

    llm = ChatVertexAI(model_name=eval_model_name, credentials=creds)
    ragas_vertexai_llm = LangchainLLMWrapper(llm)

    for m in metrics:
      m.__setattr__("llm", ragas_vertexai_llm)

      if hasattr(m, "embeddings"):
          m.__setattr__("embeddings", embedding_model)

    return metrics

In [ ]:
def setLLMRAGModel(creds, model_name: str, max_tokens: int, temp: float):
    system = (
        """You are a helpful medical assistant who always tries to answer a question in Spanish in a detailed and professional manner based on the following context:

          {context}

          Your answer can be a variation of the context, such as a conclusion, medical diagnoses (if possible using context), comparison, paraphrase, or modify the wording to fit the question (such as being more informal or more scientific) etc. But you can't add additional information, please don't try, kindly reply that you can't respond with the information you currently have.
          If the question has no relation to your provided context, kindly mention that you cannot answer that question, but do not mention anything about the context."""
    )

    human = "{text}"
    prompt = ChatPromptTemplate.from_messages([("system", system), ("human", human)])

    chat = ChatVertexAI(
        model_name=model_name,
        max_tokens=max_tokens,
        credentials=creds,
        temperature = temp,
        safety_settings = {
          HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: HarmBlockThreshold.BLOCK_NONE,
          HarmCategory.HARM_CATEGORY_HATE_SPEECH: HarmBlockThreshold.BLOCK_NONE,
          HarmCategory.HARM_CATEGORY_HARASSMENT: HarmBlockThreshold.BLOCK_NONE,
          HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: HarmBlockThreshold.BLOCK_NONE,
        },
    )

    chain = prompt | chat

    return chain

# Database 📊

## Imports

In [ ]:
from langchain.document_loaders import DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.schema import Document
from langchain_community.embeddings.sentence_transformer import SentenceTransformerEmbeddings
from langchain.vectorstores import Chroma

import chromadb
from chromadb.config import Settings
import unicodedata

## Code

In [ ]:
def splitTextInChunks(documents: list[Document], dictionary: dict) -> list:
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=2300,
        chunk_overlap=10,
        length_function=len,
        add_start_index=True,
    )

    chunks = text_splitter.split_documents(documents)
    for chunk in chunks:
        title = unicodedata.normalize('NFC', chunk.metadata['source'].split("/")[-1].split(".")[0])
        chunk.metadata['also_called'] = ', '.join(str(elemento) for elemento in dictionary[title]['also_called'])
        chunk.metadata['related_topic'] = ', '.join(str(elemento) for elemento in dictionary[title]['related_topic'])
        chunk.metadata['url'] = dictionary[title]['url']

    print(f"Split {len(documents)} documents into {len(chunks)} chunks.")

    return chunks

In [ ]:
def setVectorialDB(dictionary: dict):
    loader = DirectoryLoader(DATA_PATH, glob="*.md")
    documents = loader.load()

    chunks = splitTextInChunks(documents, dictionary)

    db = Chroma.from_documents(
        chunks, embedding_function, persist_directory=CHROMA_PATH
    )

    db.persist()
    print(f"Saved {len(chunks)} chunks to {CHROMA_PATH}.")

# RAG 🤖

## Imports

In [ ]:
import pickle
import google.auth

## Code

In [ ]:
class llm()
  def __init__(llm,llm):
    self.llm = llm

In [ ]:
class RAG():
  def __init__(llm,database):
    self.llm = llm
    self.database = database

  def RAG(query_text: str, metadata: bool):

In [ ]:
def RAG(query_text: str, metadata: bool):
    results = db.similarity_search_with_relevance_scores(query_text, k=3)
    if len(results) == 0 or results[0][1] < 0.60:
        print(f"Lo lamento, pero no cuento con esa información en mi base de datos.")
        return None,None

    context_text = "\n\n---\n\n".join([doc.page_content.replace('\n', ' ').replace('  ', ' ') for doc, _score in results])

    message = chain.invoke(
        {
            "context": context_text,
            "text": query_text,
        }
    )

    if len(message.content) == 0:
      print("¡Ups! Parece que hubo un error al responder, inténtalo de nuevo.")

    if metadata:
        print("La información con la que se responde a tu pregunta está basada en los siguientes documentos:")
        files = []
        for doc,_score in results:
            archivo = doc.metadata.get("source", None)
            if archivo not in files:
                files.append(archivo)
                otros_nombres = doc.metadata.get("also_called", None)
                temas_relacionados = doc.metadata.get("related_topic", None)
                url = doc.metadata.get("url", None)

                print("Archivo: {}".format(archivo))
                if otros_nombres:
                    print("Otros nombres: {}".format(otros_nombres))
                if temas_relacionados:
                    print("Temas relacionados: {}".format(temas_relacionados))
                print("URL: {}\n".format(url))

    return results, message.content

In [ ]:
if __name__ == "__main__":
    creds, _ = google.auth.default(quota_project_id=CLOUD_PROJECT)
    embedding_function = VertexAIEmbeddings(model_name=EMBEDDING_MODEL, credentials=creds)
    metrics = setEvalMetrics(creds, EVAL_MODEL, embedding_function);
    chain = setLLMRAGModel(creds, LLM_MODEL, 8192, 0.5)
    stop_token_ids = [0]
    with open(DICT_MEDLINE_PATH, 'rb') as f:
      medlineDict = pickle.load(f)
    setVectorialDB(medlineDict)
    db = Chroma(persist_directory=CHROMA_PATH, embedding_function=embedding_function)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


Split 1021 documents into 1483 chunks.
Saved 1483 chunks to chroma.


/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  warn_deprecated(


# Config New Dataset 📑

## Imports

In [ ]:
from datasets import Dataset
from unidecode import unidecode

## Code

In [ ]:
def createDataset(questions: list[str], ground_truths: list[str]):
    answers = []
    contexts = []

    for query in questions:
      results, answer = RAG(query, False)
      answers.append(unidecode(answer))
      contexts.append([unidecode(docs.page_content) for docs, _score in results])

    ground_truths = [unidecode(element) for element in ground_truths]
    questions = [unidecode(element) for element in questions]

    data = {
        "question": questions,
        "ground_truth": ground_truths,
        "answer": answers,
        "contexts": contexts
    }

    dataset = Dataset.from_dict(data)
    return dataset

# RAG Evaluation 📈

## Imports

In [ ]:
from ragas import evaluate
import pandas as pd

## Code

In [ ]:
def evaluateRag(dataset):
    data = []
    for n in range(0,len(dataset)):
      result = evaluate(
        dataset.select(range(n,n+1)),
        metrics = metrics,
        is_async = False,
        raise_exceptions = False
      )
      df = result.to_pandas()
      data.append(df)

    nuevo_df = pd.concat(data, ignore_index=True)
    return nuevo_df

# Evaluate RAG With Dataset📋

## Eval RAG

### Questions & Ground_truths

In [ ]:
questions = [
    "¿Cuáles son los síntomas comunes de la gripe?",
    "¿Qué debo hacer si tengo fiebre alta?",
    "¿Cuáles son los signos de una infección de oído?",
    "¿Cómo puedo prevenir resfriados y enfermedades similares?",
    "¿Cuál es la diferencia entre un resfriado y la gripe?",
    "¿Cuándo debo consultar a un médico por un dolor de garganta?",
    "¿Cómo puedo aliviar los síntomas de la alergia?",
    "¿Cómo sé si tengo una infección bacteriana?",
    "¿Qué debo hacer si creo que tengo una infección de la piel?"
    ]

ground_truths = [
    "Los síntomas comunes de la gripe incluyen fiebre, dolor de garganta, tos, congestión nasal, dolores musculares y fatiga.",
    "Si tienes fiebre alta, es importante descansar, mantenerse hidratado y tomar medicamentos como el paracetamol para reducir la fiebre.",
    "Los signos de una infección de oído pueden incluir dolor en el oído, fiebre, problemas de audición y secreción del oído.",
    "Para prevenir resfriados y enfermedades similares, lávate las manos con frecuencia, evita el contacto cercano con personas enfermas y mantén un estilo de vida saludable.",
    "La gripe tiende a tener síntomas más graves y puede llevar a complicaciones como neumonía, mientras que un resfriado generalmente es más leve y se centra en la nariz y la garganta.",
    "Debes consultar a un médico por un dolor de garganta si dura más de una semana, si tienes dificultad para tragar o respirar, o si tienes fiebre alta.",
    "Para aliviar los síntomas de la alergia, evita los alérgenos conocidos, usa medicamentos antihistamínicos y considera la terapia de alergia si los síntomas son graves.",
    "Las señales de advertencia de una infección bacteriana pueden incluir fiebre alta persistente, enrojecimiento, hinchazón, dolor intenso y secreción de pus.",
    "Si crees que tienes una infección de la piel, lava el área afectada con agua y jabón suave y aplica una crema antibiótica de venta libre. Si no mejora, consulta a un médico."
]

### Evaluation

In [ ]:
dataset = createDataset(questions, ground_truths)
df = evaluateRag(dataset)
df

Evaluating:   0%|          | 0/4 [00:00<?, ?it/s]

Evaluating:   0%|          | 0/4 [00:00<?, ?it/s]

Evaluating:   0%|          | 0/4 [00:00<?, ?it/s]

Evaluating:   0%|          | 0/4 [00:00<?, ?it/s]

Evaluating:   0%|          | 0/4 [00:00<?, ?it/s]

Evaluating:   0%|          | 0/4 [00:00<?, ?it/s]

Evaluating:   0%|          | 0/4 [00:00<?, ?it/s]

Evaluating:   0%|          | 0/4 [00:00<?, ?it/s]

Evaluating:   0%|          | 0/4 [00:00<?, ?it/s]

,question,ground_truth,answer,contexts,answer_relevancy,context_recall,context_precision,harmfulness
0,?Cuales son los sintomas comunes de la gripe?,Los sintomas comunes de la gripe incluyen fieb...,Los sintomas de la gripe aparecen repentinamen...,"[Gripe\n\n?Que es la gripe?\n\nLa gripe, tambi...",0.654095,1.0,1.000000,1
1,?Que debo hacer si tengo fiebre alta?,"Si tienes fiebre alta, es importante descansar...","Si tiene fiebre alta, debe consultar a un prof...",[Fiebre\n\nLa fiebre es una temperatura del cu...,0.758466,1.0,1.000000,1
2,?Cuales son los signos de una infeccion de oido?,Los signos de una infeccion de oido pueden inc...,Los signos de una infeccion de oido incluyen: ...,[Infecciones de los oidos\n\nLas infecciones d...,0.724758,1.0,1.000000,1
3,?Como puedo prevenir resfriados y enfermedades...,Para prevenir resfriados y enfermedades simila...,Aunque no existe una vacuna para prevenir el r...,[Tomar medicamentos de venta libre para el dol...,0.798250,1.0,0.333333,1
4,?Cual es la diferencia entre un resfriado y la...,La gripe tiende a tener sintomas mas graves y ...,La gripe y el resfriado son causados por difer...,"[Gripe\n\n?Que es la gripe?\n\nLa gripe, tambi...",0.850575,1.0,1.000000,1
5,?Cuando debo consultar a un medico por un dolo...,Debes consultar a un medico por un dolor de ga...,Debes consultar a un medico si el dolor de gar...,[Dolor de garganta\n\nSu garganta es un tubo q...,0.694266,1.0,1.000000,1
6,?Como puedo aliviar los sintomas de la alergia?,"Para aliviar los sintomas de la alergia, evita...","Para aliviar los sintomas de alergia, puedes s...",[Alergia\n\nUna alergia es una reaccion de su ...,0.727615,1.0,1.000000,0
7,?Cuales son las senales de advertencia de una ...,Las senales de advertencia de una infeccion ba...,Los sintomas de una infeccion bacteriana varia...,[Infecciones por campylobacter\n\nLa infeccion...,0.715924,0.0,0.000000,1
8,?Que debo hacer si creo que tengo una infeccio...,"Si crees que tienes una infeccion de la piel, ...","Si cree que tiene una infeccion de la piel, de...","[Para diagnosticar una infeccion de la piel, s...",0.762685,1.0,0.500000,1


In [ ]:
dataset = createDataset(questions, ground_truths)
df = evaluateRag(dataset)
df

Evaluating:   0%|          | 0/4 [00:00<?, ?it/s]

Evaluating:   0%|          | 0/4 [00:00<?, ?it/s]

Evaluating:   0%|          | 0/4 [00:00<?, ?it/s]

Evaluating:   0%|          | 0/4 [00:00<?, ?it/s]

Evaluating:   0%|          | 0/4 [00:00<?, ?it/s]

Evaluating:   0%|          | 0/4 [00:00<?, ?it/s]

Evaluating:   0%|          | 0/4 [00:00<?, ?it/s]

Evaluating:   0%|          | 0/4 [00:00<?, ?it/s]

Evaluating:   0%|          | 0/4 [00:00<?, ?it/s]

,question,ground_truth,answer,contexts,answer_relevancy,context_recall,context_precision,harmfulness
0,?Cuales son los sintomas comunes de la gripe?,Los sintomas comunes de la gripe incluyen fieb...,Los sintomas de la gripe aparecen repentinamen...,"[Gripe\n\n?Que es la gripe?\n\nLa gripe, tambi...",0.654095,1.0,1.000000,1
1,?Que debo hacer si tengo fiebre alta?,"Si tienes fiebre alta, es importante descansar...","Si tienes fiebre alta, es importante que consu...",[Fiebre\n\nLa fiebre es una temperatura del cu...,0.758466,1.0,1.000000,1
2,?Cuales son los signos de una infeccion de oido?,Los signos de una infeccion de oido pueden inc...,"Si un nino aun no habla, hay que prestar atenc...",[Infecciones de los oidos\n\nLas infecciones d...,0.714182,1.0,1.000000,1
3,?Como puedo prevenir resfriados y enfermedades...,Para prevenir resfriados y enfermedades simila...,Aunque no existe una vacuna para prevenir el r...,[Tomar medicamentos de venta libre para el dol...,0.834472,1.0,0.333333,1
4,?Cual es la diferencia entre un resfriado y la...,La gripe tiende a tener sintomas mas graves y ...,Aunque los sintomas del resfriado y de la grip...,"[Gripe\n\n?Que es la gripe?\n\nLa gripe, tambi...",0.822864,1.0,1.000000,1
5,?Cuando debo consultar a un medico por un dolo...,Debes consultar a un medico por un dolor de ga...,Deberias consultar a un medico si el dolor de ...,[Dolor de garganta\n\nSu garganta es un tubo q...,0.829083,1.0,1.000000,1
6,?Como puedo aliviar los sintomas de la alergia?,"Para aliviar los sintomas de la alergia, evita...","Para aliviar los sintomas de alergia, su profe...",[Alergia\n\nUna alergia es una reaccion de su ...,0.777405,1.0,1.000000,1
7,?Cuales son las senales de advertencia de una ...,Las senales de advertencia de una infeccion ba...,"Lo siento, pero no puedo responder a esa pregu...",[Infecciones por campylobacter\n\nLa infeccion...,0.000000,0.0,0.000000,1
8,?Que debo hacer si creo que tengo una infeccio...,"Si crees que tienes una infeccion de la piel, ...",Si sospecha que tiene una infeccion de la piel...,"[Para diagnosticar una infeccion de la piel, s...",0.757831,1.0,0.500000,1


In [ ]:
dataset = createDataset(questions, ground_truths)
df = evaluateRag(dataset)
df

¡Ups! Parece que hubo un error al responder, inténtalo de nuevo.


Evaluating:   0%|          | 0/4 [00:00<?, ?it/s]

Evaluating:   0%|          | 0/4 [00:00<?, ?it/s]

Evaluating:   0%|          | 0/4 [00:00<?, ?it/s]

Evaluating:   0%|          | 0/4 [00:00<?, ?it/s]

...

Evaluating:   0%|          | 0/4 [00:00<?, ?it/s]

Evaluating:   0%|          | 0/4 [00:00<?, ?it/s]

Evaluating:   0%|          | 0/4 [00:00<?, ?it/s]

Evaluating:   0%|          | 0/4 [00:00<?, ?it/s]

,question,ground_truth,answer,contexts,answer_relevancy,context_recall,context_precision,harmfulness
0,?Cuales son los sintomas comunes de la gripe?,Los sintomas comunes de la gripe incluyen fieb...,Los sintomas de la gripe aparecen repentinamen...,"[Gripe\n\n?Que es la gripe?\n\nLa gripe, tambi...",0.654095,1.0,1.000000,1
1,?Que debo hacer si tengo fiebre alta?,"Si tienes fiebre alta, es importante descansar...","Si tienes fiebre alta, es importante que consu...",[Fiebre\n\nLa fiebre es una temperatura del cu...,0.758466,1.0,1.000000,1
2,?Cuales son los signos de una infeccion de oido?,Los signos de una infeccion de oido pueden inc...,"Las infecciones de oido son comunes, especialm...",[Infecciones de los oidos\n\nLas infecciones d...,0.714182,1.0,1.000000,1
3,?Como puedo prevenir resfriados y enfermedades...,Para prevenir resfriados y enfermedades simila...,,[Tomar medicamentos de venta libre para el dol...,0.723733,1.0,0.333333,1
4,?Cual es la diferencia entre un resfriado y la...,La gripe tiende a tener sintomas mas graves y ...,Aunque los sintomas del resfriado y de la grip...,"[Gripe\n\n?Que es la gripe?\n\nLa gripe, tambi...",0.766604,1.0,1.000000,1
5,?Cuando debo consultar a un medico por un dolo...,Debes consultar a un medico por un dolor de ga...,Si tu dolor de garganta es severo o no desapar...,[Dolor de garganta\n\nSu garganta es un tubo q...,0.842613,1.0,1.000000,0
6,?Como puedo aliviar los sintomas de la alergia?,"Para aliviar los sintomas de la alergia, evita...","Para aliviar los sintomas de alergia, puedes s...",[Alergia\n\nUna alergia es una reaccion de su ...,0.770078,1.0,1.000000,0
7,?Cuales son las senales de advertencia de una ...,Las senales de advertencia de una infeccion ba...,Las infecciones bacterianas pueden causar una ...,[Infecciones por campylobacter\n\nLa infeccion...,0.693945,0.0,0.000000,1
8,?Que debo hacer si creo que tengo una infeccio...,"Si crees que tienes una infeccion de la piel, ...",Si sospecha que tiene una infeccion de la piel...,"[Para diagnosticar una infeccion de la piel, s...",0.706059,1.0,0.500000,1


# Test RAG 🥳

In [ ]:
# @title Ask Me
insert_your_question = "¿Qué es una enfermedad de transmisión sexual?" # @param {type:"string"}
result, answer = RAG(insert_your_question,True)
if answer is not None:
  print(answer)

La información con la que se responde a tu pregunta está basada en los siguientes documentos:
Archivo: /content/drive/MyDrive/Amilcar Gilberto Pérez Canto/Proyecto_OW?/Archivos/Enfermedades de transmisión sexual.md
Otros nombres: ETS, Enfermedades transmitidas sexualmente, Enfermedades venéreas
Temas relacionados: Enfermedad inflamatoria pélvica, Gonorrea, Herpes simple, Infecciones por clamidia, Sífilis, Tricomoniasis, Verrugas genitales, VIH, Virus del papiloma humano
URL: https://medlineplus.gov/spanish/sexuallytransmitteddiseases.html

Archivo: /content/drive/MyDrive/Amilcar Gilberto Pérez Canto/Proyecto_OW?/Archivos/Salud sexual del adolescente.md
Temas relacionados: Desarrollo del adolescente, Embarazo de adolescentes, Salud de los adolescentes
URL: https://medlineplus.gov/spanish/teensexualhealth.html

Archivo: /content/drive/MyDrive/Amilcar Gilberto Pérez Canto/Proyecto_OW?/Archivos/Gonorrea.md
Temas relacionados: Enfermedades de transmisión sexual
URL: https://medlineplus.

In [ ]:
# @title Ask Me
insert_your_question = "¿Cuál es la diferencia entre covid-19 y gripe?" # @param {type:"string"}
result, answer = RAG(insert_your_question,True)
if answer is not None:
  print(answer)

La información con la que se responde a tu pregunta está basada en los siguientes documentos:
Archivo: /content/drive/MyDrive/Amilcar Gilberto Pérez Canto/Proyecto_OW?/Archivos/Gripe.md
Otros nombres: Gripa, Influenza, Trancazo
Temas relacionados: Gripe aviar, Gripe H1N1 (gripe porcina), Infecciones virales, Resfrío, Vacuna para la gripe
URL: https://medlineplus.gov/spanish/flu.html

Archivo: /content/drive/MyDrive/Amilcar Gilberto Pérez Canto/Proyecto_OW?/Archivos/Enfermedad del coronavirus 2019 (COVID-19).md
Temas relacionados: Afecciones posteriores al COVID-19 (COVID-19 persistente), Pruebas para el COVID-19, Vacunas contra el COVID-19
URL: https://medlineplus.gov/spanish/covid19coronavirusdisease2019.html

Tanto el COVID-19 como la gripe son enfermedades respiratorias causadas por virus distintos, pero presentan algunas diferencias clave:

**Síntomas:**  Aunque comparten síntomas como fiebre, tos y fatiga, el COVID-19 puede causar pérdida del gusto y del olfato, algo menos común

In [ ]:
# @title Ask Me
insert_your_question = "Me gustaría empezar a hacer ejercicio, ¿Cuánto debería hacer si soy un adulto?" # @param {type:"string"}
result, answer = RAG(insert_your_question,True)
if answer is not None:
  print(answer)

La información con la que se responde a tu pregunta está basada en los siguientes documentos:
Archivo: /content/drive/MyDrive/Amilcar Gilberto Pérez Canto/Proyecto_OW?/Archivos/¿Cuánto ejercicio debo hacer?.md
URL: https://medlineplus.gov/spanish/howmuchexercisedoineed.html

Archivo: /content/drive/MyDrive/Amilcar Gilberto Pérez Canto/Proyecto_OW?/Archivos/Riesgos de una vida sedentaria.md
Otros nombres: Enfermedades por estar sentado mucho tiempo, Riesgos a la salud de una vida inactiva
URL: https://medlineplus.gov/spanish/healthrisksofaninactivelifestyle.html

Como adulto, debería intentar hacer al menos 150 minutos de actividad aeróbica de intensidad moderada o 75 minutos de actividad aeróbica de alta intensidad cada semana. 

También puede hacer una combinación de ambas. Intente distribuir esta cantidad de ejercicio a lo largo de la semana, en lugar de hacerlo todo en uno o dos días. 

Además del ejercicio aeróbico, intente realizar actividades de fortalecimiento muscular dos ve

In [ ]:
# @title Ask Me
insert_your_question = "Tengo fiebre, tos y fatiga, pero también problemas para respirar y diarrea, ¿Qué enfermedad podría tener?" # @param {type:"string"}
result, answer = RAG(insert_your_question,True)
if answer is not None:
  print(answer)

La información con la que se responde a tu pregunta está basada en los siguientes documentos:
Archivo: /content/drive/MyDrive/Amilcar Gilberto Pérez Canto/Proyecto_OW?/Archivos/Neumonía.md
Otros nombres: Pulmonía
Temas relacionados: Enfermedad del coronavirus 2019 (COVID-19), Enfermedad del legionario, Infecciones por Haemophilus, Infecciones por neumococo, Infecciones por Pneumocystis
URL: https://medlineplus.gov/spanish/pneumonia.html

Archivo: /content/drive/MyDrive/Amilcar Gilberto Pérez Canto/Proyecto_OW?/Archivos/Gastroenteritis.md
Otros nombres: Virus estomacal
Temas relacionados: Diarrea, Enfermedades del intestino delgado, Enfermedades transmitidas por alimentos, Náusea y vómitos, Rotavirus, Salud del viajero
URL: https://medlineplus.gov/spanish/gastroenteritis.html

Archivo: /content/drive/MyDrive/Amilcar Gilberto Pérez Canto/Proyecto_OW?/Archivos/Problemas respiratorios.md
Otros nombres: Disnea, Problemas para respirar
Temas relacionados: Apnea del sueño, Asma, Enfermeda

In [ ]:
# @title Ask Me
insert_your_question = "¿Cómo sé si tengo una infección bacteriana?" # @param {type:"string"}
result, answer = RAG(insert_your_question,True)
if answer is not None:
  print(answer)

La información con la que se responde a tu pregunta está basada en los siguientes documentos:
Archivo: /content/drive/MyDrive/Amilcar Gilberto Pérez Canto/Proyecto_OW?/Archivos/Infecciones bacterianas.md
Otros nombres: Infecciones causadas por bacterias
Temas relacionados: Antibióticos, Botulismo, Celulitis, Enfermedad de Lyme, Infecciones meningocócicas, Infecciones por clamidia, Infecciones por Clostridium difficile, Infecciones por Escherichia coli, Infecciones por estafilococo, Infecciones por estreptococo, Infecciones por micobacterias, Infecciones por neumococo, Infecciones por salmonela, Medidas para el control de infecciones, Picaduras de garrapatas, Salud del viajero, Tosferina, Tuberculosis, Tétanos, Vacunación
URL: https://medlineplus.gov/spanish/bacterialinfections.html

Archivo: /content/drive/MyDrive/Amilcar Gilberto Pérez Canto/Proyecto_OW?/Archivos/Enfermedades infecciosas.md
Otros nombres: Infecciones
Temas relacionados: Antibióticos, Enfermedades parasitarias, Infec

In [ ]:
# @title Ask Me
insert_your_question = "¿Qué es la clamidia y qué síntomas presenta?" # @param {type:"string"}
result, answer = RAG(insert_your_question,True)
if answer is not None:
  print(answer)

La información con la que se responde a tu pregunta está basada en los siguientes documentos:
Archivo: /content/drive/MyDrive/Amilcar Gilberto Pérez Canto/Proyecto_OW?/Archivos/Infecciones por clamidia.md
Otros nombres: Clamidia
Temas relacionados: Enfermedades de transmisión sexual
URL: https://medlineplus.gov/spanish/chlamydiainfections.html

La clamidia es una infección de transmisión sexual (ITS) común causada por la bacteria Chlamydia trachomatis.  

Muchas personas con clamidia no tienen síntomas, por lo que pueden no saber que están infectadas. Sin embargo, incluso sin síntomas, la clamidia puede causar problemas de salud graves si no se trata. 

Cuando sí se presentan síntomas, pueden aparecer varias semanas después de la infección. 

**En las mujeres, los síntomas pueden incluir:**

* Flujo vaginal anormal, que puede tener un olor fuerte
* Sensación de ardor al orinar
* Dolor abdominal bajo (si la infección se propaga)
* Dolor durante las relaciones sexuales (si la infección 